In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'  # Disable oneDNN warnings

import tensorflow as tf
tf.get_logger().setLevel('ERROR')  # Only show errors, not info/warnings
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import numpy as np

In [ ]:
## basic deep autoencoder arhitecture

# Encoder: multiple layers for more complex representations
encoded = Dense(128, activation='relu')
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

# Decoder: symmetric layers to reconstruct the input
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

# Deep autoencoder model
autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

ValueError: Only input tensors may be passed as positional arguments. The following argument value should be passed as a keyword argument: <Dense name=dense_1, built=False> (of type <class 'keras.src.layers.core.dense.Dense'>)